1. Started with: Pre-trained LLM model(meta-llama/Llama-3.3-70B-Instruct:fireworks-ai)
2. Applied: Fine-tuning on mental health datasets(none)
3. Built: RAG architecture with mental health knowledge base
4. Used: Prompt engineering to guide empathetic responses
5. Included: System prompts for safety and behavior guidelines

# pre-existing generic mrntal-health chatbot


LLM-based chatbots (newer generation - 22 total identified):

MindGuide,
ChatCounselor, 
Replika,
PsyChat,
SMILE,
Psy-LLM,
Assistant-Instruct,
CBT-LLM,
MindWatch,
EmLLM,
Counsellor Chatbot

# data resource

1. Beyond Blue: https://www.beyondblue.org.au/mental-health/resource-library
2. Reachout: https://au.reachout.com/challenges-and-coping/abuse-and-violence/sexual-assault-support-services
3. First aid: https://www.mhfa.com.au/resources-support
4. CCI: https://www.cci.health.wa.gov.au/Resources/Looking-After-Others4




=========================================================================

CBT, or Cognitive Behavioural Therapy,


=========================================================================

ICD-11 data (Profession)

DSM-5-TR


=========================================================================




# Metrics

Technical Metrics (Computer-Based)
Most common metrics:

1. Perplexity: How well the model predicts responses (lower = better)
2. ROUGE-L: Measures if responses match expected answers
3. BLEU scores: Checks precision of language generation
4. Distinct-1/2/3: Measures response variety (can it say things differently?)
5. BERTScore: Captures semantic meaning (does it understand context?)
6. Empathy%: What percentage of responses show compassion?

Human Evaluation Metrics (Expert/User Assessment)
General metrics:

1. Helpfulness, Fluency, Relevance, Logic
2. Informativeness, Understanding, Consistency, Coherence, Empathy, Expertise, Engagement

Counseling-specific metrics:

1. Direct Guidance, Approval and Reassurance, Restatement, Reflection, Listening, Interpretation, Self-disclosure
(These mirror what real therapists do)

Reliability check:

1. Krippendorff's Alpha - ensures different evaluators agree



# scrapping DSM-5-TR clinical case

In [ ]:
import pandas as pd  # Kept as in original, though not used here
import time
import random
import json
import requests
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
import undetected_chromedriver as uc  # Import the undetectable version

# Set up options for undetectable Chrome
options = uc.ChromeOptions()
options.add_argument('--disable-blink-features=AutomationControlled')
options.add_argument('--user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/118.0.0.0 Safari/537.36')  # Example user-agent

# Enable performance logging to capture network requests
options.set_capability("goog:loggingPrefs", {'performance': 'ALL'})

# Initialize the undetectable driver
driver = uc.Chrome(options=options)

try:
    # Navigate to the URL (https://psychiatryonline.org/doi/epub/10.1176/appi.books.9781615375295)
    driver.get('https://psychiatryonline.org/doi/epdf/10.1176/appi.books.9781615375295')
    
    # Wait for the PDF viewer to load
    WebDriverWait(driver, 60).until(EC.presence_of_element_located((By.ID, 'viewer')))
    
    # Simulate human scroll to load more content (e.g., scroll down a bit)
    actions = ActionChains(driver)
    actions.move_by_offset(random.randint(100, 300), random.randint(100, 300)).perform()  # Random mouse move
    time.sleep(random.uniform(1, 3))  # Random delay
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight / 2);")  # Scroll halfway
    time.sleep(random.uniform(2, 5))  # Wait like a human reading
    
    # If login is required, manually log in via the browser window, then press Enter here
    # input("Log in if necessary and press Enter to continue...")
    
    # Optional: More scrolls to ensure all pages load if lazy-loaded
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(random.uniform(3, 6))
    
    # Hide webdriver property (though undetected_chromedriver handles most of this)
    driver.execute_script("Object.defineProperty(navigator, 'webdriver', {get: () => undefined})")
    
    # Give some time for network requests to complete
    time.sleep(5)
    
    # Get performance logs
    logs = driver.get_log('performance')
    
    pdf_url = None
    for entry in logs:
        message = json.loads(entry['message'])['message']
        if message['method'] == 'Network.responseReceived':
            response = message['params'].get('response', {})
            url = response.get('url', '').lower()
            if '.pdf' in url or 'application/pdf' in response.get('mimeType', ''):
                pdf_url = response['url']
                print(f"Found potential PDF URL: {pdf_url}")
                break  # Assuming the first PDF is the main one; adjust if multiple
    
    if not pdf_url:
        raise Exception("No PDF URL found in network logs. Try scrolling more or increasing wait time.")
    
    # Download the PDF using requests with cookies from the browser session
    cookies = {c['name']: c['value'] for c in driver.get_cookies()}
    headers = {'User-Agent': options.arguments[-1].split('=')[1]}  # Reuse user-agent
    response = requests.get(pdf_url, cookies=cookies, headers=headers)
    
    if response.status_code == 200:
        with open('downloaded.pdf', 'wb') as f:
            f.write(response.content)
        print("PDF downloaded successfully as 'downloaded.pdf'")
    else:
        raise Exception(f"Failed to download PDF: Status code {response.status_code}")

finally:
    driver.quit()


In [ ]:




import PyPDF2
import csv
import re

# def extract_text_from_pdf(pdf_path):
#     with open(pdf_path, 'rb') as file:
#         reader = PyPDF2.PdfReader(file)
#         text = ''
#         for page in reader.pages:
#             text += page.extract_text() + '\n'
#     return text


# def write_to_csv(cases, csv_path='output_cases.csv'):
#     with open(csv_path, 'w', newline='', encoding='utf-8') as file:
#         writer = csv.DictWriter(file, fieldnames=['case', 'description', 'discussion', 'diagnosis'])
#         writer.writeheader()
#         writer.writerows(cases)
#     print(f'CSV file created at: {csv_path}')

# # Usage
# pdf_path = './rag_system/dsm5/DSM-5-TR clinical case.pdf'  # Replace with your PDF path
# text = extract_text_from_pdf(pdf_path)
# # save the text to a txt file for checking
# with open('extracted_text.txt', 'w', encoding='utf-8') as f:
#     f.write(text)

import csv
import re


def remove_names(text):
    """Remove names for privacy protection"""
    # Common name patterns in case studies
    # Pattern 1: Names followed by age (e.g., "Ashley, age 17" or "Brandon was a 12-year-old")
    text = re.sub(r'\b[A-Z][a-z]+,?\s+(?:age\s+\d+|was\s+a\s+\d+-year-old)', '[PATIENT]', text)
    
    # Pattern 2: Capitalized names at the beginning of sentences or after periods
    # But preserve common medical terms and abbreviations
    medical_terms = r'(?!DSM|ASD|ADHD|OCD|IQ|PhD|MD|DO|MA)'
    text = re.sub(rf'\b{medical_terms}([A-Z][a-z]+)(?:\'s)?\s+(?:mother|father|parents|family|sister|brother|teacher)', r'[PATIENT]\'s \2', text)
    
    # Pattern 3: Possessive names (e.g., "Ashley's mother", "Brandon's father")
    text = re.sub(r'\b[A-Z][a-z]+\'s\s+(?:mother|father|parents|family|sister|brother|teacher|doctor)', r'[PATIENT]\'s', text)
    
    # Pattern 4: Names in subject position (harder to detect, more conservative)
    # Only remove if followed by specific verbs or patterns common in case studies
    text = re.sub(r'\b[A-Z][a-z]+\s+(?:was|had|did|made|spoke|walked|responded|attended|received|presented)', '[PATIENT]', text)
    
    return text

def clean_text(text):
    """Remove extra whitespace and empty lines, put all text in one line"""
    # Fix spaced-out words (e.g., "p s y c h i a t r y" -> "psychiatry")
    # Look for pattern: single letter + space + single letter (repeated)
    while re.search(r'\b(\w) (\w) (\w)', text):
        text = re.sub(r'\b(\w) (?=\w )', r'\1', text)
    
    # Replace multiple spaces with single space
    text = re.sub(r' +', ' ', text)
    
    # Remove empty lines and join everything with a space
    lines = [line.strip() for line in text.split('\n') if line.strip()]
    return ' '.join(lines)

def remove_suggested_readings(text):
    """Remove Suggested Readings section from text"""
    # Remove everything from "Suggested Readings" onwards
    text = re.sub(r'Suggested Reading.*', '', text, flags=re.DOTALL | re.IGNORECASE)
    return text.strip()

def parse_cases_from_file(txt_path):
    """Parse cases from extracted text file"""
    with open(txt_path, 'r', encoding='utf-8') as f:
        text = f.read()
    
    # Remove all Suggested Readings sections before processing
    text = re.sub(
        r'Suggested Reading.*?(?=CASE \d+\.\d+|\Z)',
        '',
        text,
        flags=re.DOTALL | re.IGNORECASE
    )
    
    # Split by CASE pattern
    case_pattern = r'(CASE \d+\.\d+)'
    parts = re.split(case_pattern, text)
    
    cases = []
    
    # Process pairs: case_number, content
    for i in range(1, len(parts), 2):
        if i + 1 >= len(parts):
            break
            
        case_num = parts[i].strip()
        content = parts[i + 1].strip()
        
        # Split content into lines
        lines = content.split('\n')
        
        # Extract case title (first non-empty line)
        case_title = ''
        author_line_idx = 0
        for idx, line in enumerate(lines):
            line = line.strip()
            if line:
                case_title = line
                author_line_idx = idx
                break
        
        # Skip author lines (usually 1-3 lines after title with names ending in degree)
        desc_start_idx = author_line_idx + 1
        while desc_start_idx < len(lines):
            line = lines[desc_start_idx].strip()
            # Check if line looks like an author (contains Ph.D., M.D., etc.)
            if not line or re.search(r'\b(Ph\.?D\.?|M\.?D\.?|M\.?A\.?|D\.?O\.?)\b', line):
                desc_start_idx += 1
            else:
                break
        
        # Rejoin content from description start
        remaining_content = '\n'.join(lines[desc_start_idx:])
        
        # Remove any remaining Suggested Readings
        # remaining_content = remove_suggested_readings(remaining_content)
        
        # Extract description (everything before "Discussion")
        description = ''
        discussion = ''
        diagnosis = ''
        
        # Split by "Discussion" first
        if 'Discussion' in remaining_content:
            parts_disc = re.split(r'Discussion\s*\n', remaining_content, maxsplit=1)
            description = parts_disc[0].strip()
            after_discussion = parts_disc[1].strip() if len(parts_disc) > 1 else ''
            
            # Now split after_discussion by "Diagnosis" or "Diagnoses"
            diag_match = re.search(r'(Diagnos[ei]s?)\s*\n', after_discussion)
            
            if diag_match:
                # Extract discussion (before "Diagnosis")
                discussion = after_discussion[:diag_match.start()].strip()
                
                # Extract diagnosis (after "Diagnosis/Diagnoses")
                after_diagnosis = after_discussion[diag_match.end():].strip()
                
                # First, handle line continuations with hyphens
                after_diagnosis = re.sub(r'-\s*\n\s*', '', after_diagnosis)
                
                # Then handle regular line breaks within bullet points
                after_diagnosis = re.sub(r'(?<!•)\n(?!•)', ' ', after_diagnosis)
                
                # Clean up the diagnosis to keep only bullet points
                diag_lines = []
                for line in after_diagnosis.split('\n'):
                    line = line.strip()
                    if line.startswith('•'):
                        diag_lines.append(line)
                diagnosis = ' '.join(diag_lines)
            else:
                # No diagnosis section found, everything after Discussion is discussion
                discussion = after_discussion.strip()
                diagnosis = ''
        else:
            # No Discussion section found
            description = remaining_content.strip()
            discussion = ''
            diagnosis = ''
        
        cases.append({
            # {case_num} - 
            'case': f"{case_title}",
            'description': clean_text(description),
            'discussion': clean_text(discussion),
            'diagnosis': clean_text(diagnosis)
        })
    
    return cases

def write_to_csv(cases, csv_path='output_cases.csv'):
    """Write cases to CSV file"""
    with open(csv_path, 'w', newline='', encoding='utf-8') as file:
        writer = csv.DictWriter(file, fieldnames=['case', 'description', 'discussion', 'diagnosis'])
        writer.writeheader()
        writer.writerows(cases)
    print(f'CSV file created successfully: {csv_path}')
    print(f'Total cases extracted: {len(cases)}')

# Main execution
if __name__ == "__main__":
    txt_path = 'extracted_text.txt'
    
    try:
        cases = parse_cases_from_file(txt_path)
        write_to_csv(cases)
        
        # # Print summary
        # for i, case in enumerate(cases, 1):
        #     print(f"\n--- Case {i} ---")
        #     print(f"Title: {case['case'][:50]}...")
        #     print(f"Description length: {len(case['description'])} chars")
        #     print(f"Discussion length: {len(case['discussion'])} chars")
        #     print(f"Diagnosis length: {len(case['diagnosis'])} chars")
            
    except FileNotFoundError:
        print(f"Error: Could not find {txt_path}")
    except Exception as e:
        print(f"Error: {e}")
        import traceback
        traceback.print_exc()

CSV file created successfully: output_cases.csv
Total cases extracted: 104


In [3]:
import platform
import psutil

# Install psutil if not already: pip install psutil

# Get OS details
print("Operating System:", platform.system())
print("OS Version:", platform.version())
print("OS Release:", platform.release())

# Get CPU details
print("\nCPU Info:")
print("Processor:", platform.processor())
print("Number of CPU Cores:", psutil.cpu_count(logical=True))

# Get RAM details
print("\nRAM Info:")
ram = psutil.virtual_memory()
print("Total RAM:", f"{ram.total / (1024 ** 3):.2f} GB")
print("Available RAM:", f"{ram.available / (1024 ** 3):.2f} GB")

# Get Disk details (for C: drive on Windows)
print("\nDisk Info (C: drive):")
disk = psutil.disk_usage('C:\\')
print("Total Disk Space:", f"{disk.total / (1024 ** 3):.2f} GB")
print("Used Disk Space:", f"{disk.used / (1024 ** 3):.2f} GB")
print("Free Disk Space:", f"{disk.free / (1024 ** 3):.2f} GB")

# Get GPU details (requires NVIDIA GPU and CUDA installed)
try:
    import torch
    if torch.cuda.is_available():
        print("\nGPU Info:")
        print("GPU Name:", torch.cuda.get_device_name(0))
        print("GPU VRAM:", f"{torch.cuda.get_device_properties(0).total_memory / (1024 ** 3):.2f} GB")
    else:
        print("\nNo GPU detected or CUDA not available.")
except ImportError:
    print("\nTorch not installed; skipping GPU check.")


import faiss
print(faiss.get_num_gpus())

Operating System: Windows
OS Version: 10.0.26100
OS Release: 10

CPU Info:
Processor: Intel64 Family 6 Model 158 Stepping 10, GenuineIntel
Number of CPU Cores: 12

RAM Info:
Total RAM: 15.84 GB
Available RAM: 2.57 GB

Disk Info (C: drive):
Total Disk Space: 441.52 GB
Used Disk Space: 235.69 GB
Free Disk Space: 205.83 GB

GPU Info:
GPU Name: NVIDIA GeForce GTX 1650 Ti
GPU VRAM: 4.00 GB
1


# import the pre-trained model

In [4]:
# # Install main packages (your original, but with -U for upgrades)
# %pip install -U transformers bitsandbytes accelerate peft datasets

# # Install LangChain and FAISS (use faiss-gpu-cu12 for GPU; fallback to faiss-cpu if issues)
# %pip install langchain-community faiss-gpu-cu12 sentence-transformers

# Optional: Downgrade PyArrow if conflicts arise (e.g., if cudf errors pop up during imports)
# !pip install pyarrow==18.0.0 --force-reinstall


In [1]:
import os
import numpy as np
import pandas as pd
import re
from sentence_transformers import SentenceTransformer
import faiss
from faiss import read_index, write_index
import torch
import logging
import nltk
from openai import OpenAI  # Add this import for API

# Download VADER lexicon if not already downloaded
# nltk.download('vader_lexicon', quiet=True)

# Set up logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Detect device (still needed for embedder, but not for LLM)
device = 'cuda' if torch.cuda.is_available() else 'cpu'
logger.info(f"Using device: {device}")

# Set up OpenAI client for Hugging Face API router
client = OpenAI(
    base_url="https://router.huggingface.co/v1",
    api_key=os.environ["HF_TOKEN"],
)
model_name = "meta-llama/Llama-3.3-70B-Instruct:fireworks-ai"  # Use a larger model via API

INFO:__main__:Using device: cuda


# General DATA PREP

In [3]:
import os
import numpy as np
import pandas as pd
from sentence_transformers import SentenceTransformer
import faiss
# from PyPDF2 import PdfReader  # Or import pdfplumber for better extraction
import pdfplumber  # Or import pdfplumber for better extraction
import logging
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer

# Download VADER lexicon if not already downloaded
nltk.download('vader_lexicon', quiet=True)

# Set up logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Detect device
import torch
device = 'cuda' if torch.cuda.is_available() else 'cpu'
logger.info(f"Using device: {device}")


# "cambridgeltl/SapBERT-from-PubMedBERT-fulltext"
# Load embedder
embedder = SentenceTransformer('cambridgeltl/SapBERT-from-PubMedBERT-fulltext', device=device)




# Configurable paths (change category as needed)
category = 'general'  # 'anxiety', 'depression', 'ptsd', 'suicide', general
pdf_dir = f'./rag_system/{category}/'
rag_chunks_csv_path = f'./rag_system/{category}/chunks.csv'
rag_embeddings_path = f'./rag_system/{category}/chunk_embeddings.npy'
rag_index_path = f'./rag_system/{category}/faiss_index.index'

# Ensure directories exist
os.makedirs(os.path.dirname(rag_chunks_csv_path), exist_ok=True)

# Function to extract and chunk text from a single PDF
def extract_and_chunk_pdf(pdf_path, chunk_size=300, overlap=40):
    chunks = []
    try:
        with pdfplumber.open(pdf_path) as pdf:
            full_text = ""
            for page_num, page in enumerate(pdf.pages, start=1):
                text = page.extract_text() or ""  # Use layout=True if needed: page.extract_text(layout=True)
                full_text += f"\n\n[Page {page_num}]\n{text}"

        # Simple chunking: Split by words with overlap
        words = full_text.split()
        for i in range(0, len(words), chunk_size - overlap):
            chunk_words = words[i:i + chunk_size]
            chunk_text = " ".join(chunk_words)
            chunks.append({
                'content': chunk_text,
                'source_pdf': os.path.basename(pdf_path),
                'page_start': (i // chunk_size) + 1,  # Approximate page
                'category': category
            })

        logger.info(f"Extracted {len(chunks)} chunks from {pdf_path}")

    except Exception as e:
        logger.error(f"Error processing {pdf_path}: {e}")
    return chunks

# ==================================================================

# Step 1: Process all PDFs and save chunks to CSV
if os.path.exists(rag_chunks_csv_path):
    logger.info(f"Loading existing chunks from {rag_chunks_csv_path}")
    chunks_df = pd.read_csv(rag_chunks_csv_path)
else:
    all_chunks = []
    for filename in os.listdir(pdf_dir):
        if filename.endswith('.pdf'):
            pdf_path = os.path.join(pdf_dir, filename)
            pdf_chunks = extract_and_chunk_pdf(pdf_path)
            all_chunks.extend(pdf_chunks)

    if not all_chunks:
        raise ValueError(f"No chunks extracted from PDFs in {pdf_dir}")
    
    chunks_df = pd.DataFrame(all_chunks)
    chunks_df['chunk_id'] = range(len(chunks_df))  # Add unique ID
    chunks_df.to_csv(rag_chunks_csv_path, index=False)
    logger.info(f"Saved {len(chunks_df)} chunks to {rag_chunks_csv_path}")

# Add sentiment if not present
if 'sentiment' not in chunks_df.columns:
    logger.info("Computing sentiment for chunks...")
    sia = SentimentIntensityAnalyzer()
    
    def get_sentiment(text):
        score = sia.polarity_scores(text)['compound']
        if score > 0.05:
            return 'positive'
        elif score < -0.05:
            return 'negative'
        else:
            return 'neutral'
    
    chunks_df['sentiment'] = chunks_df['content'].apply(get_sentiment)
    chunks_df.to_csv(rag_chunks_csv_path, index=False)  # Resave with sentiment
    logger.info("Sentiment added and CSV updated.")



# Step 2: Load or compute embeddings
if os.path.exists(rag_embeddings_path):
    logger.info("Loading precomputed embeddings...")
    chunk_embeddings_np = np.load(rag_embeddings_path)
else:
    logger.info("Computing embeddings...")
    chunk_contents = chunks_df['content'].tolist()
    chunk_embeddings = embedder.encode(
        chunk_contents,
        batch_size=128,
        show_progress_bar=True,
        convert_to_tensor=True,
        device=device,
        normalize_embeddings=True  # Built-in normalization
    )
    chunk_embeddings_np = chunk_embeddings.cpu().numpy()
    np.save(rag_embeddings_path, chunk_embeddings_np)
    logger.info(f"Embeddings saved to {rag_embeddings_path}")

# Step 3: Build/Load FAISS Index
d = chunk_embeddings_np.shape[1]  # Embedding dimension
if os.path.exists(rag_index_path):
    logger.info("Loading existing FAISS index...")
    faiss_index = faiss.read_index(rag_index_path)
else:
    logger.info("Building FAISS index...")
    index = faiss.IndexFlatIP(d)  # Inner product for normalized vectors
    index.add(chunk_embeddings_np)
    faiss.write_index(index, rag_index_path)
    logger.info(f"FAISS index saved to {rag_index_path}")




INFO:__main__:Using device: cuda
INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: cambridgeltl/SapBERT-from-PubMedBERT-fulltext
INFO:__main__:Loading existing chunks from ./rag_system/general/chunks.csv
INFO:__main__:Loading precomputed embeddings...
INFO:__main__:Loading existing FAISS index...


# ICD-11 Data prep


In [4]:
import requests
import json
from dotenv import load_dotenv
load_dotenv()



# Configurable paths for ICD-11 data
icd_dir = './rag_system/ICD-11_Data/'
icd_json_path = f'{icd_dir}/icd_disorders.json'
icd_chunks_csv_path = f'{icd_dir}/icd_chunks.csv'
icd_embeddings_path = f'{icd_dir}/icd_embeddings.npy'
icd_index_path = f'{icd_dir}/faiss_index.index'

# Ensure directories exist
os.makedirs(icd_dir, exist_ok=True)

# ICD-11 API settings (assumes environment variables for credentials)
CLIENT_ID = os.environ.get('ICD_CLIENT_ID')
CLIENT_SECRET = os.environ.get('ICD_CLIENT_SECRET')
SCOPE = 'icdapi_access'
GRANT_TYPE = 'client_credentials'
TOKEN_ENDPOINT = 'https://icdaccessmanagement.who.int/connect/token'
API_BASE = 'https://id.who.int/icd'
RELEASE = '2025-01'  # Update to the latest release if needed
LINEARIZATION = 'mms'  # Mortality and Morbidity Statistics
CHAPTER_ENTITY_ID = '334423054'  # Entity ID for Chapter 6: Mental, behavioural or neurodevelopmental disorders

# Function to get OAuth token
def get_access_token():
    if not CLIENT_ID or not CLIENT_SECRET:
        raise ValueError("ICD_CLIENT_ID and ICD_CLIENT_SECRET must be set in environment variables.")
    
    payload = {
        'client_id': CLIENT_ID,
        'client_secret': CLIENT_SECRET,
        'scope': SCOPE,
        'grant_type': GRANT_TYPE
    }
    response = requests.post(TOKEN_ENDPOINT, data=payload)
    response.raise_for_status()
    return response.json()['access_token']

# Function to fetch entity data
def fetch_entity(entity_id, token, is_foundation=False):
    headers = {
        'Authorization': f'Bearer {token}',
        'Accept': 'application/json',
        'Accept-Language': 'en',  # Can change for other languages
        'API-Version': 'v2'
    }
    if is_foundation:
        url = f'{API_BASE}/entity/{entity_id}'
    else:
        url = f'{API_BASE}/release/11/{RELEASE}/{LINEARIZATION}/{entity_id}'
    response = requests.get(url, headers=headers)
    response.raise_for_status()
    return response.json()

# Recursive function to collect all disorders under a parent entity
def collect_disorders(parent_id, token, collected=[], depth=0, max_depth=10):
    if depth > max_depth:
        logger.warning(f"Max depth reached for {parent_id}")
        return collected
    
    try:
        data = fetch_entity(parent_id, token)
        
        # Extract relevant fields
        code = data.get('code', '')
        fully_specified_name = data.get('title', {}).get('@value', '') if isinstance(data.get('title'), dict) else data.get('title', '')
        description = data.get('definition', {}).get('@value', '') if isinstance(data.get('definition'), dict) else data.get('definition', '')
        
        # Exclusions
        exclusions = []
        if 'exclusion' in data:
            for excl in data['exclusion']:
                label = excl.get('label', {}).get('@value', '') if isinstance(excl.get('label'), dict) else excl.get('label', '')
                exclusions.append(label)
        exclusions_str = '; '.join(exclusions)
        
        # Index Terms
        index_terms = []
        if 'indexTerm' in data:
            for term in data['indexTerm']:
                label = term.get('label', {}).get('@value', '') if isinstance(term.get('label'), dict) else term.get('label', '')
                index_terms.append(label)
        index_terms_str = '; '.join(index_terms)
        
        # Check if this is a leaf node (no children)
        has_children = 'child' in data and data['child']
        
        # Only collect if it's a leaf node, has a code, and is not a chapter/block
        if not has_children and code and fully_specified_name and not fully_specified_name.startswith('Chapter') and not fully_specified_name.startswith('Block'):
            collected.append({
                'code': code,
                'fully_specified_name': fully_specified_name,
                'description': description,
                'exclusions': exclusions_str,
                'all_index_terms': index_terms_str,
                'entity_id': parent_id
            })
            logger.info(f"Collected (leaf): {code} - {fully_specified_name}")
        
        # Recurse on children if present
        if has_children:
            for child_uri in data['child']:
                child_id = child_uri.split('/')[-1]  # Extract ID from URI
                collect_disorders(child_id, token, collected, depth + 1, max_depth)
    
    except requests.HTTPError as e:
        logger.error(f"Error fetching {parent_id}: {e}")
    
    return collected



# ========================================================================



# Step 1: Fetch and save ICD-11 data to CSV and JSON
if os.path.exists(icd_chunks_csv_path):
    logger.info(f"Loading existing ICD data from {icd_chunks_csv_path}")
    icd_df = pd.read_csv(icd_chunks_csv_path)
elif os.path.exists(icd_json_path):
    logger.info(f"Loading existing ICD data from {icd_json_path}")
    with open(icd_json_path, 'r') as f:
        disorders = json.load(f)
    icd_df = pd.DataFrame(disorders)
    icd_df.to_csv(icd_chunks_csv_path, index=False)
    logger.info(f"Saved CSV from JSON to {icd_chunks_csv_path}")
else:
    token = get_access_token()
    disorders = collect_disorders(CHAPTER_ENTITY_ID, token)
    
    if not disorders:
        raise ValueError("No disorders collected from ICD-11 API.")
    
    with open(icd_json_path, 'w') as f:
        json.dump(disorders, f, indent=4)
    logger.info(f"Saved disorders to {icd_json_path}")
    
    icd_df = pd.DataFrame(disorders)
    icd_df.to_csv(icd_chunks_csv_path, index=False)
    logger.info(f"Saved {len(icd_df)} disorders to {icd_chunks_csv_path}")

# Add sentiment if not present (though may not be as relevant for clinical data, but for consistency)
if 'sentiment' not in icd_df.columns:
    logger.info("Computing sentiment for ICD entries...")
    sia = SentimentIntensityAnalyzer()
    
    def get_sentiment(text):
        score = sia.polarity_scores(text)['compound']
        if score > 0.05:
            return 'positive'
        elif score < -0.05:
            return 'negative'
        else:
            return 'neutral'
    
    # Concatenate fields for sentiment
    icd_df['content'] = icd_df.apply(lambda row: f"{row['code']} {row['fully_specified_name']} {row['description']} {row['exclusions']} {row['all_index_terms']}", axis=1)
    icd_df['sentiment'] = icd_df['content'].apply(get_sentiment)
    icd_df.to_csv(icd_chunks_csv_path, index=False)
    logger.info("Sentiment added and CSV updated.")
else:
    # Ensure content column exists
    icd_df['content'] = icd_df.apply(lambda row: f"{row['code']} {row['fully_specified_name']} {row['description']} {row['exclusions']} {row['all_index_terms']}", axis=1)



# Step 2: Load or compute embeddings
if os.path.exists(icd_embeddings_path):
    logger.info("Loading precomputed embeddings...")
    chunk_embeddings_np = np.load(icd_embeddings_path)
else:
    logger.info("Computing embeddings...")
    chunk_contents = icd_df['content'].tolist()
    chunk_embeddings = embedder.encode(
        chunk_contents,
        batch_size=128,
        show_progress_bar=True,
        convert_to_tensor=True,
        device=device,
        normalize_embeddings=True
    )
    chunk_embeddings_np = chunk_embeddings.cpu().numpy()
    np.save(icd_embeddings_path, chunk_embeddings_np)
    logger.info(f"Embeddings saved to {icd_embeddings_path}")

# Step 3: Build/Load FAISS Index
if os.path.exists(icd_index_path):
    logger.info("Loading existing FAISS index...")
    faiss_index = faiss.read_index(icd_index_path)
else:
    logger.info("Building FAISS index...")
    d = chunk_embeddings_np.shape[1]
    index = faiss.IndexFlatIP(d)
    index.add(chunk_embeddings_np)
    write_index(index, icd_index_path)
    logger.info(f"FAISS index saved to {icd_index_path}")

INFO:__main__:Loading existing ICD data from ./rag_system/ICD-11_Data//icd_chunks.csv
INFO:__main__:Loading precomputed embeddings...
INFO:__main__:Loading existing FAISS index...


# Load RAG data

In [5]:




# loaded resources for all
# Categories (assuming lowercase for paths, but adjust if needed)
# \'anxiety', 'depression', 'ptsd', 'suicide', 
categories = ['general']  # Include 'general' as fallback

# Load category-specific resources into a dictionary
category_resources = {}
for cat in categories:
    chunks_csv = f'./rag_system/{cat}/chunks.csv'
    emb_path = f'./rag_system/{cat}/chunk_embeddings.npy'
    idx_path = f'./rag_system/{cat}/faiss_index.index'
    
    if os.path.exists(chunks_csv) and os.path.exists(emb_path) and os.path.exists(idx_path):
        chunks_df = pd.read_csv(chunks_csv)
        # Ensure sentiment is present (fallback compute if missing)
        if 'sentiment' not in chunks_df.columns:
            logger.info(f"Computing sentiment for {cat} chunks...")
            sia = SentimentIntensityAnalyzer()
            def get_sentiment(text):
                score = sia.polarity_scores(text)['compound']
                if score > 0.05: return 'positive'
                elif score < -0.05: return 'negative'
                else: return 'neutral'
            chunks_df['sentiment'] = chunks_df['content'].apply(get_sentiment)
            chunks_df.to_csv(chunks_csv, index=False)
            logger.info(f"Sentiment added for {cat} and CSV updated.")
        
        chunk_emb_np = np.load(emb_path)
        cat_index = read_index(idx_path)
        category_resources[cat] = {
            'df': chunks_df,
            'embeddings': chunk_emb_np,
            'index': cat_index
        }
        logger.info(f"Loaded resources for category: {cat}")
    else:
        logger.warning(f"Missing resources for category: {cat}. Skipping.")


# load ICD-11 resources

icd_dir = './rag_system/ICD-11_Data/'
icd_chunks_csv_path = f'{icd_dir}/icd_chunks.csv'
icd_embeddings_path = f'{icd_dir}/icd_embeddings.npy'
icd_index_path = f'{icd_dir}/faiss_index.index'



if os.path.exists(icd_chunks_csv_path):
    icd_df = pd.read_csv(icd_chunks_csv_path)
    if 'content' not in icd_df.columns:
        icd_df['content'] = icd_df.apply(lambda row: f"{row['code']} {row['fully_specified_name']} {row['description']} {row['exclusions']} {row['all_index_terms']}", axis=1)
        icd_df.to_csv(icd_chunks_csv_path, index=False)
else:
    raise FileNotFoundError(f"ICD-11 CSV not found at {icd_chunks_csv_path}. Please run the ICD-11 prep script first.")

if os.path.exists(icd_embeddings_path):
    icd_embeddings_np = np.load(icd_embeddings_path)
else:
    raise FileNotFoundError(f"ICD-11 embeddings not found at {icd_embeddings_path}.")

if os.path.exists(icd_index_path):
    icd_index = read_index(icd_index_path)
else:
    raise FileNotFoundError(f"ICD-11 FAISS index not found at {icd_index_path}.")


INFO:__main__:Loaded resources for category: general


# building the RAG system

In [ ]:


# API-based generation for RAG (single prompt)
def generate_with_llm_rag(prompt: str, max_tokens: int = 120):
    messages = [{"role": "user", "content": prompt}]
    completion = client.chat.completions.create(
        model=model_name,
        messages=messages,
        max_tokens=max_tokens,
        temperature=0.8,
        top_p=0.95,
        frequency_penalty=1.2  # Approximate repetition_penalty with frequency_penalty
    )
    return completion.choices[0].message.content.strip()



# Category retrieval with method - refined to use detected category
def category_retrieve(category: str, query: str, method: str = 'original', k: int = 10) -> list[dict]:  # Retrieve more for rerank
    if category not in category_resources:
        logger.warning(f"Category '{category}' not found. Falling back to 'general'.")
        category = 'general'
        if category not in category_resources:
            raise ValueError("No 'general' resources available as fallback.")
    
    res = category_resources[category]
    chunks_df = res['df']
    cat_index = res['index']
    
    queries = [query]
    
    if method == 'multi':
        prompt = f"""
            Strictly follow: Output EXACTLY 5 unique variant queries similar to '{query}', one per line.
            Preserve key elements (events, causes, feelings).
            No reasoning, no steps, no examples, no introductions, no extra text at all.
            Directly start with the first query.

            Example output format (do not include this in output):
            Variant1
            Variant2
            Variant3
            Variant4
            Variant5
            """
        variants_response = generate_with_llm_rag(prompt)
        variants = [line.strip() for line in variants_response.split('\n') if line.strip() and len(line.split()) > 2 and not re.match(r'^(#|Step|Example|For reference|The|To|Output|Preserve|No|Directly)', line, re.I)]
        queries = list(set(v for v in variants if v.lower() != query.lower()))[:5]
        if not queries or len(queries) < 3:  # Fallback if poor generation
            queries = [query, f"{query} coping strategies", f"{query} emotional support"]
        logger.info(f"Generated multi-queries: {queries}")

    elif method == 'hyde':
        prompt = f"""
            Generate a concise hypothetical answer document for the query '{query}'.
            Output only the document text, without any introductions, explanations, numbering,
            or extra formatting.
        """
        hyde_response = generate_with_llm_rag(prompt)
        # Clean up: Join lines into a single document string, remove leading/trailing whitespace, and strip common prefixes like "Document:"
        hyde_doc = ' '.join(line.strip() for line in hyde_response.split('\n') if line.strip()).replace('Document:', '').strip()
        queries = [hyde_doc]  # Use the cleaned doc as "query" for embedding
        logger.info(f"Generated HyDE document: {hyde_doc[:200]}...")
    
    # Embed all queries
    query_embs = embedder.encode(queries, convert_to_tensor=True, device=device, normalize_embeddings=True).cpu().numpy()
    
    # Search for each, collect unique results with max score
    all_results = {}
    for q_emb in query_embs:
        q_emb = q_emb.reshape(1, -1)
        distances, indices = cat_index.search(q_emb, k=k)
        for i in range(len(distances[0])):
            idx = indices[0][i]
            if idx == -1: continue
            score = distances[0][i]
            if idx not in all_results or score > all_results[idx]['score']:
                row = chunks_df.iloc[idx]
                all_results[idx] = {
                    'content': row['content'],
                    'source_pdf': row['source_pdf'],
                    'page_start': row['page_start'],
                    'sentiment': row['sentiment'],
                    'score': score
                }
    
    # Get top k by score first (before rerank)
    sorted_results = sorted(all_results.values(), key=lambda x: x['score'], reverse=True)[:k]
    return sorted_results

# Rerank function
def rerank_results(results: list[dict]) -> list[dict]:
    sentiment_order = {'positive': 0, 'neutral': 1, 'negative': 2}
    # Sort by sentiment order, then by score descending
    sorted_results = sorted(results, key=lambda x: (sentiment_order.get(x['sentiment'], 3), -x['score']))
    return sorted_results[:5]  # Top 5 after rerank

# Full workflow - refined to always use top similar post for category
def full_rag_workflow(query: str, method: str = 'original') -> tuple[list[dict], dict]:
    

    category = 'general'  # Comment out if you want dynamic category

    # Step 2: Retrieve from category-specific database with method
    retrieved = category_retrieve(category, query, method=method)
    
    # Step 3: Rerank
    reranked = rerank_results(retrieved)
    return reranked

# API-based generation for chatbot (messages list)
def generate_with_llm(messages: list[dict], max_tokens: int = 150) -> str:
    completion = client.chat.completions.create(
        model=model_name,
        messages=messages,
        max_tokens=max_tokens,
        temperature=0.8,
        top_p=0.95,
        frequency_penalty=1.2  # Approximate repetition_penalty
    )
    return completion.choices[0].message.content.strip()








# example_query = "my dog pass away"

# # Test with different methods: 'original', 'multi', 'hyde'
# results = full_rag_workflow(example_query, method='hyde')  # Change method as needed
# for i, res in enumerate(results):
#     print(f"Top {i+1}: Score: {res['score']:.4f}, Sentiment: {res['sentiment']}, Source: {res['source_pdf']}")
#     print(f"Content snippet: {res['content'][:300]}...\n")




# Prompt engineering

In [ ]:
# Refined chatbot loop with history and token management
def run_chatbot(method: str = 'original', max_context_tokens: int = 8192, max_history_pairs: int = 10):
    
    system_prompt = """
        You are a compassionate and empathetic mental health assistant. Your responses should:
        - Analyze the user's input with care and understanding.
        - Respond directly in a comforting manner, tailoring to the specific query, history, and provided guidelines.
        - Incorporate relevant details from guidelines naturally (e.g., coping strategies).
        - Offer gentle, practical coping suggestions where relevant, varying them based on context.
        - Keep the tone warm, supportive, and conversational—vary phrasing to avoid repetition.
        - You may suggest possible matching disorders from ICD-11 if provided in the prompt, but emphasize that it's not a formal diagnosis and professional consultation is needed.
        - Responses should be 60-100 words.
        """
    # - Suggest professional help if needed (e.g., helplines like beyondblue: 1300 22 4636).
    
    history = []  # List of dicts: [{'role': 'user', 'content': ...}, {'role': 'assistant', 'content': ...}]
    
    print("Welcome to the Mental Health Chatbot. Type 'quit' to exit.")
    
    while True:
        try:
            query = input("You: ").strip()
            if query.lower() in ['quit', 'exit']:
                print("Chatbot: Goodbye! Take care.")
                break
            
            if not query:
                print("Chatbot: Please enter a message.")
                continue



            # Accumulate user inputs from history + current query
            user_inputs = [h['content'] for h in history if h['role'] == 'user'] + [query]
            accumulated_input = ' '.join(user_inputs)

            # Embed accumulated input
            query_embedding = embedder.encode(accumulated_input, convert_to_tensor=True, device=device, normalize_embeddings=True)
            query_embedding_np = query_embedding.cpu().numpy().reshape(1, -1)



            # Search ICD-11 index for top 4 matches
            distances, indices = icd_index.search(query_embedding_np, k=4)
            icd_infos = []
            high_score_disorders = []
            if len(distances[0]) > 0:
                for i in range(len(distances[0])):
                    score = distances[0][i]
                    idx = indices[0][i]
                    if idx == -1: continue
                    if score > 0.5:
                        row = icd_df.iloc[idx]
                        disorder_name = row['fully_specified_name']
                        code = row['code']
                        symptoms = row['description']
                        icd_infos.append(f"""
                            Disorder Name: {disorder_name}
                            Disorder Code: {code}
                            Disorder symptoms: {symptoms}
                            """)
                        if score > 0.7:
                            high_score_disorders.append(f"{disorder_name} ({code})")
            
            icd_prompt_section = ""
            if icd_infos:
                icd_prompt_section = f"\n\nPossible matching disorders from ICD-11 (similarity scores > 0.5):\n{''.join(icd_infos)}\nRemember, this is not a diagnosis; suggest professional help."





            
            # Get RAG results
            retrieved_docs= full_rag_workflow(query, method=method)
            
            # Format retrieved docs as context string
            doc_context = "\n".join([f"Doc {i+1} (Sentiment: {doc['sentiment']}, Score: {doc['score']:.4f}): {doc['content']}" for i, doc in enumerate(retrieved_docs)])
            
            
            # Build user prompt
            user_prompt = f"""
                    User's message: {query}

                    Additional relevant guidelines (use these to inform suggestions if they fit the query):
                    {doc_context}

                    Respond empathetically, drawing from guidelines for specific ideas. If the query involves loss or grief, suggest personalized memorials or support resources. Vary your language from previous responses.
                    """ + icd_prompt_section
            


            
            # Build messages
            messages = [{"role": "system", "content": system_prompt}] + history + [{"role": "user", "content": user_prompt}]
            
            # Check token length and truncate history proactively if needed
            # Note: Without local tokenizer, approximate token count or skip; for simplicity, assume API handles it or implement a rough estimator
            # For now, we'll skip precise truncation; API has context limits (e.g., 128k for Llama-3.3)
            def estimate_tokens(text: str) -> int:
                return len(text.split()) * 1.3 + 100  # Rough estimate + buffer

            while sum(estimate_tokens(msg['content']) for msg in messages) > max_context_tokens:
                if len(history) <= 2:
                    logger.warning("Context exceeds limit; proceeding.")
                    break
                history = history[2:]  # Remove oldest pair
                messages = [{"role": "system", "content": system_prompt}] + history + [{"role": "user", "content": user_prompt}]
                logger.info(f"Truncated history to {len(history)} entries.")
            
            # Generate response
            response = generate_with_llm(messages, max_tokens=130)

            # If any with similarity > 0.7, append diagnosis suggestion
            if high_score_disorders:
                suggestions = ', '.join(high_score_disorders)
                response += f"\n\nBased on the provided information, your symptoms suggest possible diagnoses of {suggestions} according to ICD-11."
            
            print("Chatbot:", response)
            
            # Append to history
            history.append({"role": "user", "content": query})
            history.append({"role": "assistant", "content": response})
        
        except Exception as e:
            logger.error(f"Error in chatbot loop: {e}")
            print("Chatbot: Sorry, something went wrong. Please try again.")



run_chatbot(method='multi')  # Change method as needed: 'original', 'multi', 'hyde'

Welcome to the Mental Health Chatbot. Type 'quit' to exit.


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

d:\miniconda3\envs\chat_env\lib\site-packages\transformers\models\bert\modeling_bert.py:412: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:455.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(
INFO:httpx:HTTP Request: POST https://router.huggingface.co/v1/chat/completions "HTTP/1.1 200 OK"
INFO:__main__:Generated multi-queries: ['my name is stormy weather', 'i am named after a rainy day', 'raindrop is what my parents call me', 'my given name means rainy night']


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

INFO:httpx:HTTP Request: POST https://router.huggingface.co/v1/chat/completions "HTTP/1.1 200 OK"


Chatbot: Hi Rain, it's lovely to meet you. I'm here to listen and support you in any way I can. Your name is a beautiful reminder of the beauty and calm that can be found in life. If you're feeling overwhelmed, remember that it's okay to take things one step at a time. Consider reaching out to Beyond Blue (1300 22 4636) for guidance and support.
Chatbot: Goodbye! Take care.


# Evaluation

In [22]:
import pandas as pd
import numpy as np
from sklearn.metrics import precision_score
import re
from rouge_score import rouge_scorer
from bert_score import score as bert_score
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from nltk.util import ngrams
from collections import Counter
import torch

# Load your DSM-5 test data (adjust path)
test_df = pd.read_csv('./rag_system/dsm5/dsm5_testing_data.csv')  # Columns: case, description, discussion, diagnosis

# Example: Create test queries from description + discussion
test_queries = test_df['description'] + ' ' + test_df['discussion']
ground_truth_diagnoses = test_df['diagnosis'].tolist()  # Use as relevance labels or references

# Helper for semantic relevance (cosine similarity using your embedder)
def compute_semantic_relevance(query, retrieved_contents):
    query_emb = embedder.encode([query], convert_to_tensor=True, device=device, normalize_embeddings=True)
    retrieved_embs = embedder.encode(retrieved_contents, convert_to_tensor=True, device=device, normalize_embeddings=True)
    similarities = torch.nn.functional.cosine_similarity(query_emb, retrieved_embs).cpu().numpy()
    return np.mean(similarities)  # Average relevance score

# For RAG ground truth: Assume a chunk is "relevant" if it contains keywords from diagnosis (simple heuristic; improve with manual labels if possible)
def is_relevant(chunk_content, gt_diagnosis):
    keywords = set(re.findall(r'\w+', gt_diagnosis.lower()))
    chunk_words = set(re.findall(r'\w+', chunk_content.lower()))
    overlap = len(keywords & chunk_words) / len(keywords) if keywords else 0
    return overlap > 0.5  # Threshold; adjust as needed

# Evaluation for RAG Retrieval system

In [24]:
from sklearn.metrics import ndcg_score

K = 5
rag_metrics = {'precision': [], 'mrr': [], 'ndcg': [], 'semantic_relevance': []}

for idx, query in enumerate(test_queries[:50]):  # Batch size; adjust
    gt_diagnosis = ground_truth_diagnoses[idx]
    
    # Run your RAG retrieval (adjust method as needed, e.g., 'multi')
    retrieved_docs = full_rag_workflow(query, method='multi')
    retrieved_contents = [doc['content'] for doc in retrieved_docs[:K]]  # Top-K after rerank
    
    # Compute binary relevance labels (for precision/MRR)
    binary_relevances = []
    # Compute graded relevance labels (for nDCG) as overlap ratios (floats)
    graded_relevances = []
    for content in retrieved_contents:
        keywords = set(re.findall(r'\w+', gt_diagnosis.lower()))
        chunk_words = set(re.findall(r'\w+', content.lower()))
        overlap = len(keywords & chunk_words) / len(keywords) if keywords else 0.0
        binary_relevances.append(1 if overlap > 0.5 else 0)  # Binary based on threshold
        graded_relevances.append(overlap)  # Graded as float overlap score
    
    # Precision@K
    prec = sum(binary_relevances) / K if retrieved_contents else 0
    rag_metrics['precision'].append(prec)
    
    # MRR
    first_relevant_rank = next((i+1 for i, rel in enumerate(binary_relevances) if rel > 0), 0)
    mrr = 1 / first_relevant_rank if first_relevant_rank > 0 else 0
    rag_metrics['mrr'].append(mrr)
    
    # nDCG@K (true relevances: assume ideal is sorted graded relevances)
    ideal_relevances = sorted(graded_relevances, reverse=True)[:K]
    ndcg = ndcg_score([ideal_relevances], [graded_relevances], k=K) if graded_relevances else 0
    rag_metrics['ndcg'].append(ndcg)
    
    # Semantic Relevance
    sem_rel = compute_semantic_relevance(query, retrieved_contents) if retrieved_contents else 0
    rag_metrics['semantic_relevance'].append(sem_rel)

# Aggregate and display
rag_results = {metric: np.mean(values) for metric, values in rag_metrics.items()}
print("RAG Evaluation Results:")
pd.DataFrame([rag_results]).T  # Display as table

INFO:httpx:HTTP Request: POST https://router.huggingface.co/v1/chat/completions "HTTP/1.1 200 OK"
INFO:__main__:Generated multi-queries: ['A 17-year-old girl with a history of developmental delays, facial dysmorphology, and possible static', 'A young woman with intellectual developmental disorder and autism spectrum disorder struggles to adapt to changes in her schedule and environment, leading to irritability and self-harm.', "A family seeks clarification on the impact of their daughter's genetic diagnosis on her long-standing diagnoses of autism and intellectual disability, as well as the risk to their other children.", 'Ashley, a 17-year-old with Kleefstra syndrome, undergoes reevaluation for autism and intellectual disability after her family discovers she has a rare genetic defect.']


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

INFO:httpx:HTTP Request: POST https://router.huggingface.co/v1/chat/completions "HTTP/1.1 200 OK"
INFO:__main__:Generated multi-queries: ['A 10-year-old boy with autism spectrum disorder was taken to a psychiatrist due to increasing tantrums and declining school performance after being bullied by classmates.', 'A 12-year-old boy with symptoms of social communication deficits and fixated interests was brought to a psychiatric evaluation by his mother for temper tantrums and declining school performance.', 'Brandon, a 12-year-old boy, exhibited signs of autism spectrum disorder, including difficulty forming friendships and fixated interests in toy cars, which contributed to his declining academic performance.', 'A young boy struggling with social interactions and having restricted interests was evaluated for psychiatric treatment due to worsening behavior']


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

INFO:httpx:HTTP Request: POST https://router.huggingface.co/v1/chat/completions "HTTP/1.1 402 Payment Required"


APIStatusError: Error code: 402 - {'error': 'You have exceeded your monthly included credits for Inference Providers. Subscribe to PRO to get 20x more monthly included credits.'}

# Evaluation for Mental heal Chatbot

In [ ]:
scorer = rouge_scorer.RougeScorer(['rouge1', 'rougeL'], use_stemmer=True)
smoothing = SmoothingFunction().method4  # For BLEU
chatbot_metrics = {'rouge1': [], 'rougeL': [], 'bleu': [], 'bert_f1': [], 'distinct1': [], 'distinct2': []}
generated_responses = []  # Collect for diversity across all

for idx, query in enumerate(test_queries[:50]):
    ref = ground_truth_diagnoses[idx]  # Reference text
    
    # Simulate chatbot: Build messages (adapt from your run_chatbot; no history for isolated eval)
    system_prompt = "Your system prompt here..."  # Copy from your code
    user_prompt = f"User's message: {query}"  # Simplified; add RAG if needed
    messages = [{"role": "system", "content": system_prompt}, {"role": "user", "content": user_prompt}]
    response = generate_with_llm(messages, max_tokens=130)
    generated_responses.append(response)
    
    # ROUGE
    rouge_scores = scorer.score(ref, response)
    chatbot_metrics['rouge1'].append(rouge_scores['rouge1'].fmeasure)
    chatbot_metrics['rougeL'].append(rouge_scores['rougeL'].fmeasure)
    
    # BLEU (sentence-level)
    ref_tokens = [re.findall(r'\w+', ref)]
    resp_tokens = re.findall(r'\w+', response)
    bleu = sentence_bleu(ref_tokens, resp_tokens, smoothing_function=smoothing)
    chatbot_metrics['bleu'].append(bleu)
    
    # BERTScore (uses BERT embeddings for semantic F1)
    _, _, bert_f1 = bert_score([response], [ref], lang='en', rescale_with_baseline=True)
    chatbot_metrics['bert_f1'].append(bert_f1.mean().item())

# Diversity across all generated responses
all_text = ' '.join(generated_responses)
for n in [1, 2]:
    grams = list(ngrams(re.findall(r'\w+', all_text), n))
    unique_grams = len(set(grams))
    total_grams = len(grams)
    diversity = unique_grams / total_grams if total_grams > 0 else 0
    chatbot_metrics[f'distinct{n}'].append(diversity)  # Average not needed; single value per batch

# Aggregate and display (diversity is batch-level)
chatbot_results = {metric: np.mean(values) for metric, values in chatbot_metrics.items()}
print("Chatbot Response Evaluation Results:")
pd.DataFrame([chatbot_results]).T  # Display as table